# State lumping with rewards

This notebook demonstrates state lumping techniques for coalescent models with reward transformations.

State lumping reduces the complexity of phase-type distributions by grouping states according to some equivalence relation while preserving the distribution of interest through appropriate reward vectors.

In [ ]:
import sys
sys.path.insert(0, '../../../src')

import numpy as np
import matplotlib.pyplot as plt
from phasic import Graph
from scipy.special import comb
import itertools
from collections import Counter, defaultdict
import h5py

# Plotting utilities

In [ ]:
def plot_sfs(sfs_values, sample_size):
    """Plot site frequency spectrum"""
    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(1, len(sfs_values) + 1)
    ax.bar(x, sfs_values, color=plt.cm.viridis(x / len(sfs_values)))
    ax.set_xlabel('Number of descendants')
    ax.set_ylabel('Expected total branch length')
    ax.set_title(f'Site Frequency Spectrum (n={sample_size})')
    plt.tight_layout()
    return fig, ax

def plot_exp_mat(exp_mat, sample_size):
    """Plot expectation matrix for two-locus model"""
    fig, ax = plt.subplots(figsize=(8, 7))
    im = ax.imshow(exp_mat, cmap='RdBu_r', aspect='auto')
    ax.set_xlabel('Locus 2 descendants')
    ax.set_ylabel('Locus 1 descendants')
    ax.set_title(f'Two-locus expectations (n={sample_size})')
    plt.colorbar(im, ax=ax)
    plt.tight_layout()
    return fig, ax

# Graph construction

## Standard coalescent

Build the standard Kingman coalescent model where the state tracks the number of lineages.

In [ ]:
def standard_coalescent(n):
    """Construct standard coalescent graph for n samples
    
    State: [n_lineages]
    Transitions: coalescence at rate n*(n-1)/2
    """
    def callback(state):
        n_lin = state[0]
        if n_lin <= 1:
            return []
        
        rate = n_lin * (n_lin - 1) / 2
        next_state = np.array([n_lin - 1], dtype=int)
        return [(next_state, rate)]
    
    graph = Graph(
        state_length=1,
        callback=callback,
        nr_samples=n
    )
    
    return graph

## Block coalescent

The block coalescent is equivalent to the standard coalescent but demonstrates state lumping principles.
In this simple case, we directly track the number of lineages.

In [ ]:
def block_coalescent(n):
    """Block coalescent - lumped version of standard coalescent
    
    This is mathematically identical to standard_coalescent but demonstrates
    the lumping concept where we group states by total lineage count.
    """
    return standard_coalescent(n)

# Site Frequency Spectrum (SFS) rewards

## SFS reward vectors

For the coalescent, the SFS is computed using reward vectors that track the total branch length
leading to exactly k descendant lineages.

In [ ]:
def get_single_locus_block_rewards(sample_size, n_lineages):
    """Get reward vector for SFS computation
    
    For a state with n_lineages, the reward for ton (time of n) is
    the number of ways to choose ton-1 lineages from n_lineages-1 lineages
    to be ancestral to the mutation.
    
    Args:
        sample_size: Number of samples
        n_lineages: Number of lineages in current state
    
    Returns:
        Reward vector of length sample_size-1
    """
    rewards = np.zeros(sample_size - 1)
    
    if n_lineages <= 1:
        return rewards
    
    for ton in range(1, sample_size):
        if ton < n_lineages:
            # Number of ways to place mutation such that it appears in exactly 'ton' samples
            rewards[ton - 1] = comb(n_lineages - 1, ton - 1, exact=True)
    
    return rewards

## Compute SFS using rewards

In [ ]:
# Build coalescent graph
sample_size = 5
graph = block_coalescent(sample_size)

# Get number of vertices
n_vertices = graph.vertices_length()
print(f"Graph has {n_vertices} vertices")

# Build reward matrix: each column is reward vector for one ton value
reward_matrix = np.zeros((n_vertices, sample_size - 1))

# For each vertex, compute reward vector
for i in range(n_vertices):
    vertex = graph.vertex_at(i)
    state = vertex.state()
    n_lineages = state[0]
    
    # Get reward vector for this state
    rewards = get_single_locus_block_rewards(sample_size, n_lineages)
    reward_matrix[i, :] = rewards

print(f"Reward matrix shape: {reward_matrix.shape}")

In [ ]:
# Compute SFS by computing expectations for each ton
sfs = np.zeros(sample_size - 1)

for ton_idx in range(sample_size - 1):
    reward_vector = reward_matrix[:, ton_idx]
    transformed = graph.reward_transform(reward_vector)
    sfs[ton_idx] = transformed.expectation()

print(f"\nSFS for n={sample_size}:")
for ton, value in enumerate(sfs, 1):
    print(f"  {ton} descendants: {value:.4f}")

# Plot SFS
plot_sfs(sfs, sample_size)
plt.show()

## Verify theoretical SFS

The theoretical SFS for the standard coalescent is E[T_i] = 1/i where i is the number of descendants.

In [ ]:
# Theoretical SFS
theoretical_sfs = 1.0 / np.arange(1, sample_size)

print("\nComparison with theoretical values:")
print("ton | Computed | Theoretical | Difference")
print("-" * 50)
for ton in range(1, sample_size):
    diff = abs(sfs[ton-1] - theoretical_sfs[ton-1])
    print(f"{ton:3d} | {sfs[ton-1]:8.4f} | {theoretical_sfs[ton-1]:11.4f} | {diff:.2e}")

# Two-locus model and lumping

For two-locus models, the state space becomes very large. We can use state lumping to reduce complexity
while preserving the distributions of interest through reward vectors.

## Two-locus state representation

For two loci with recombination, lineages can have different ancestral configurations at each locus.
We track:
- Number of lineages that are ancestral at locus 1 only
- Number of lineages that are ancestral at locus 2 only  
- Number of lineages that are ancestral at both loci

In [ ]:
def construct_two_locus_arg(sample_size, N=1.0, R=0.01):
    """Construct two-locus ARG with recombination
    
    State: Vector indexed by (n_l1, n_l2, n_linked) configurations
    - Coalescence rate: n*(n-1)/2 per pair of lineages
    - Recombination rate: R per linked lineage
    
    Args:
        sample_size: Number of samples
        N: Effective population size (default 1.0)
        R: Scaled recombination rate (default 0.01)
    
    Returns:
        Graph object
    """
    # For simplicity, we use a lumped state representation
    # State: [n_locus1, n_locus2, n_linked]
    state_length = 3
    
    def callback(state):
        n_l1_only = state[0]  # Locus 1 only
        n_l2_only = state[1]  # Locus 2 only
        n_linked = state[2]   # Both loci
        
        total_l1 = n_l1_only + n_linked
        total_l2 = n_l2_only + n_linked
        
        if total_l1 <= 1 and total_l2 <= 1:
            return []
        
        transitions = []
        
        # Recombination: linked -> separate
        if n_linked > 0:
            rate = R * n_linked
            next_state = state.copy()
            next_state[0] += 1  # New locus 1 only lineage
            next_state[1] += 1  # New locus 2 only lineage
            next_state[2] -= 1  # One less linked
            transitions.append((next_state, rate))
        
        # Coalescence within linked lineages
        if n_linked >= 2:
            rate = n_linked * (n_linked - 1) / 2
            next_state = state.copy()
            next_state[2] -= 1
            transitions.append((next_state, rate))
        
        # Coalescence: linked + locus1-only
        if n_linked > 0 and n_l1_only > 0:
            rate = n_linked * n_l1_only
            next_state = state.copy()
            next_state[0] -= 1
            transitions.append((next_state, rate))
        
        # Coalescence: linked + locus2-only
        if n_linked > 0 and n_l2_only > 0:
            rate = n_linked * n_l2_only
            next_state = state.copy()
            next_state[1] -= 1
            transitions.append((next_state, rate))
        
        # Coalescence within locus 1 only
        if n_l1_only >= 2:
            rate = n_l1_only * (n_l1_only - 1) / 2
            next_state = state.copy()
            next_state[0] -= 1
            transitions.append((next_state, rate))
        
        # Coalescence within locus 2 only
        if n_l2_only >= 2:
            rate = n_l2_only * (n_l2_only - 1) / 2
            next_state = state.copy()
            next_state[1] -= 1
            transitions.append((next_state, rate))
        
        return transitions
    
    # Initial state: all lineages linked
    graph = Graph(
        state_length=state_length,
        callback=callback,
        nr_samples=sample_size
    )
    
    # Override initial state to be [0, 0, sample_size] (all linked)
    starting_vertex = graph.starting_vertex()
    initial_state = np.array([0, 0, sample_size], dtype=int)
    first_vertex = graph.find_or_create_vertex(initial_state)
    starting_vertex.add_edge(first_vertex, 1.0)
    
    return graph

## Build and explore two-locus graph

In [ ]:
# Build small two-locus model
sample_size = 3
two_locus_graph = construct_two_locus_arg(sample_size, N=1.0, R=0.5)

n_vertices = two_locus_graph.vertices_length()
print(f"Two-locus graph has {n_vertices} vertices")

# Show some states
print("\nFirst 10 states:")
for i in range(min(10, n_vertices)):
    vertex = two_locus_graph.vertex_at(i)
    state = vertex.state()
    print(f"  Vertex {i}: [L1only={state[0]}, L2only={state[1]}, linked={state[2]}]")

## Two-locus SFS rewards

For the two-locus model, we need reward matrices that track branch lengths for each combination
of (locus1_descendants, locus2_descendants).

In [ ]:
def get_two_locus_block_rewards(sample_size, state):
    """Get reward matrix for two-locus SFS
    
    Returns a matrix where entry [i,j] is the reward when a mutation
    appears in i descendants at locus 1 and j descendants at locus 2.
    
    Args:
        sample_size: Number of samples
        state: Current state [n_l1_only, n_l2_only, n_linked]
    
    Returns:
        Reward matrix of shape (sample_size, sample_size)
    """
    n_l1_only = state[0]
    n_l2_only = state[1]
    n_linked = state[2]
    
    total_l1 = n_l1_only + n_linked
    total_l2 = n_l2_only + n_linked
    
    rewards = np.zeros((sample_size, sample_size))
    
    if total_l1 <= 1 or total_l2 <= 1:
        return rewards
    
    # For each possible (nl1, nl2) descendant count
    for nl1 in range(1, sample_size):
        for nl2 in range(1, sample_size):
            # Count configurations that lead to this (nl1, nl2)
            # This is simplified - full implementation requires more complex combinatorics
            if nl1 <= total_l1 and nl2 <= total_l2:
                # Simplified reward calculation
                rewards[nl1, nl2] = 1.0
    
    return rewards

# Precompute and cache rewards

For larger models, we can precompute reward matrices and cache them in HDF5 files for reuse.

In [ ]:
def cache_coalescent_rewards(sample_sizes, output_file='coal_block_rewards.h5'):
    """Precompute and cache SFS rewards for different sample sizes
    
    Args:
        sample_sizes: List of sample sizes to compute
        output_file: Path to HDF5 output file
    """
    with h5py.File(output_file, 'w') as h5f:
        coal_group = h5f.create_group('coalescent')
        
        for n in sample_sizes:
            print(f"Computing rewards for n={n}...")
            
            # Build graph
            graph = block_coalescent(n)
            n_vertices = graph.vertices_length()
            
            # Build reward matrix
            reward_matrix = np.zeros((n_vertices, n - 1))
            
            for i in range(n_vertices):
                vertex = graph.vertex_at(i)
                state = vertex.state()
                n_lineages = state[0]
                rewards = get_single_locus_block_rewards(n, n_lineages)
                reward_matrix[i, :] = rewards
            
            # Save to HDF5
            coal_group.create_dataset(str(n), data=reward_matrix)
    
    print(f"Saved rewards to {output_file}")

In [ ]:
# Example: cache rewards for sample sizes 3-10
# cache_coalescent_rewards(range(3, 11), 'coal_block_rewards.h5')

## Load cached rewards

In [ ]:
def load_sfs_rewards(sample_size, cache_file='coal_block_rewards.h5'):
    """Load precomputed SFS rewards from cache
    
    Args:
        sample_size: Sample size to load
        cache_file: Path to HDF5 cache file
    
    Returns:
        Reward matrix of shape (n_vertices, sample_size-1)
    """
    with h5py.File(cache_file, 'r') as h5f:
        if str(sample_size) in h5f['coalescent']:
            return h5f['coalescent'][str(sample_size)][:]
        else:
            raise ValueError(f"No cached rewards for sample size {sample_size}")

# Summary

This notebook demonstrated:

1. **Standard coalescent construction** - Building basic Kingman coalescent models
2. **State lumping** - Grouping states while preserving distributions via rewards
3. **Site Frequency Spectrum** - Computing SFS using reward transformations
4. **Two-locus models** - Extending to recombining loci with more complex state spaces
5. **Reward caching** - Precomputing and storing reward matrices for efficiency

Key principles:

- **Reward vectors** allow us to compute expectations of functions of the absorption time
- **State lumping** reduces computational complexity by grouping equivalent states
- For the SFS, rewards encode the combinatorics of mutation placement on the tree
- **Caching** precomputed rewards enables efficient reuse across analyses